In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset, MFDataset
import matplotlib.pyplot as plt
import glob
from datetime import datetime, timedelta
from scipy import stats
from sklearn.metrics import mean_squared_error
import calendar

import sys, warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
# warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [2]:
def calc_ef(lh,sh):           # Evaporative fraction
    ef = lh/(lh+sh)
    return ef.where(ef<1.2,1.2).where(ef>0.0,0.0)       # Be nice and trim outliers

def calc_lcl(t,q,ps):                             # LCL [m AGL] T in K, q in kg/kg,  ps in Pa
    eps = 0.622
    eu = q*ps/(q*(1-eps)+eps)
    td = -(243.5*np.log(eu/611.2))/(np.log(eu/611.2)-17.67) # Td in ˚C
    return 1000*(t-273.15-td)/(9.8-1.8)                 # Td in ˚C

def calc_me(t,q):             # Moist enthalpy [J/kg]
    return 1005*t+2.5e6*q     # T in K

def calc_rnet(sd,ld,su,lu):   # Net radiation at surface
    return sd-ld-su-lu

In [3]:
# dir = '/homes/eseo8/python/UFS/FLUXNET/subset_subpt/DD/FULLSET/'
dir = './FULLSET/'
site_list = sorted(glob.glob(dir+'*_DD_QC.csv'))

pts = ["P5","P6","P7a","P7","P8a","P8at"]
dirs= ["/project/UFS_land/AWS/P5_hindcast/",
       "/project/UFS_land/AWS/P6_hindcast/",
       "/project/UFS_land/AWS/P7_hindcast/P7a_",
       "/project/UFS_land/AWS/P7_hindcast/",
       "/project/UFS_land/UFS_P8_hindcast/P8a/netcdf/P8a_",
       "/project/UFS_land/UFS_P8_hindcast/P8at/netcdf/P8at_"]

######################################################################
f_start = 0 ; f_stop = len(site_list)     # These files/sites [206 stations]
######################################################################

ufs_vars = ["LCL.surface","NETRAD.surface","EF.surface","ME.surface"]

years = [x for x in range(2012,2014)]
init_sub_dates = ["0101","0401","0701","1001"]
init_dates = [None] * len(years) * len(init_sub_dates)

cnt = 0
for yr in years:
    for id in init_sub_dates:
        init_dates[cnt] = str(yr)+id
        cnt = cnt + 1

print(init_dates)
comp = dict(zlib=True, complevel=1)

['20120101', '20120401', '20120701', '20121001', '20130101', '20130401', '20130701', '20131001']


In [5]:
lon = np.empty([len(site_list)])
lat = np.empty([len(site_list)])
site = [None] * len(site_list)
igbp = [None] * len(site_list)
lon[:] = np.nan
lat[:] = np.nan

# open FluxTowers
for ff,f in enumerate(site_list[f_start:f_stop]):
    f_siteid = f.split("/")[2].split("DD")[0]
    f2 = sorted(glob.glob(dir+'/'+f_siteid+'site.csv'))
    df2 = pd.read_csv(f2[0],sep=",")
    site[ff] = df2['Site ID'].item()
    lon[ff] = df2['Longitude'].item()
    lat[ff] = df2['Latitude'].item()
    igbp[ff] = df2['IGBP type'].item()

lon = np.where(lon<0.,lon+360.,lon) # lon=[-180~180] --> [0~360]
print("station lon=({}~{}), lat=({}~{})".format(lon.min(),lon.max(),lat.min(),lat.max()))

station lon=(1.95191~357.30579), lat=(-37.4222~78.186)


In [6]:
# open UFS prototypes
for i,id in enumerate(init_dates):
    for pp,p in enumerate(pts):
        for v,vars in enumerate(ufs_vars):
            print(p,id,vars)
            if vars =="LCL.surface":
                var = ["TMP.surface","SPFH.2_m_above_ground","PRES.surface"]
                df1 = xr.open_dataset(dirs[pp]+id+'/'+var[0]+'.nc4')[var[0]].sel(longitude=lon, latitude=lat, method="nearest")
                df2 = xr.open_dataset(dirs[pp]+id+'/'+var[1]+'.nc4')[var[1]].sel(longitude=lon, latitude=lat, method="nearest")
                df3 = xr.open_dataset(dirs[pp]+id+'/'+var[2]+'.nc4')[var[2]].sel(longitude=lon, latitude=lat, method="nearest")
                
                ft = len(df1[:,0,0])
                df = df1[:ft,0,:].rename({'time1': 'time', 'longitude': 'sites'}).rename(vars).copy(deep=True)
                df[:] = np.nan
                df.coords['sites'] = site
                del df['latitude']
                
                df.attrs["short_name"] = "LCL"
                df.attrs["long_name"] = "Lifting Condensation Level"
                df.attrs["units"] = "m"
                
                for ff in range(len(lon)):
                    ft = len(df1[:,ff,ff])
                    df[:ft,ff] = calc_lcl(df1[:,ff,ff],df2[:,ff,ff],df3[:,ff,ff])
                                
                df = df.assign_coords({"lon": ("sites", lon)})
                df = df.assign_coords({"lat": ("sites", lat)})
                
                encoding = {vars: comp}
                df.to_netcdf(path='./data/'+p+'_'+id+'_'+vars+'.nc4',engine="netcdf4",unlimited_dims="time",format="netCDF4",encoding=encoding)

                del var
                del df
                del df1
                del df2
                del df3
                
            elif vars =="NETRAD.surface":
                var = ["DSWRF.surface.6_hour_ave","DLWRF.surface.6_hour_ave","USWRF.surface.6_hour_ave","ULWRF.surface.6_hour_ave"]
                df1 = xr.open_dataset(dirs[pp]+id+'/'+var[0]+'.nc4')[var[0]].sel(longitude=lon, latitude=lat, method="nearest")
                df2 = xr.open_dataset(dirs[pp]+id+'/'+var[1]+'.nc4')[var[1]].sel(longitude=lon, latitude=lat, method="nearest")
                df3 = xr.open_dataset(dirs[pp]+id+'/'+var[2]+'.nc4')[var[2]].sel(longitude=lon, latitude=lat, method="nearest")
                df4 = xr.open_dataset(dirs[pp]+id+'/'+var[3]+'.nc4')[var[3]].sel(longitude=lon, latitude=lat, method="nearest")
                
                ft = len(df1[:,0,0])
                df = df1[:ft,0,:].rename({'time1': 'time', 'longitude': 'sites'}).rename(vars).copy(deep=True)
                df[:] = np.nan
                df.coords['sites'] = site
                del df['latitude']
                
                df.attrs["short_name"] = "NETRAD"
                df.attrs["long_name"] = "Net Radiative Flux"
                df.attrs["units"] = "W/m^2"
                
                for ff in range(len(lon)):
                    ft = len(df1[:,ff,ff])
                    df[:ft,ff] = calc_rnet(df1[:,ff,ff],df2[:,ff,ff],df3[:,ff,ff],df4[:,ff,ff])
                                
                df = df.assign_coords({"lon": ("sites", lon)})
                df = df.assign_coords({"lat": ("sites", lat)})
                
                encoding = {vars: comp}
                df.to_netcdf(path='./data/'+p+'_'+id+'_'+vars+'.nc4',engine="netcdf4",unlimited_dims="time",format="netCDF4",encoding=encoding)

                del var
                del df
                del df1
                del df2
                del df3
                del df4

            elif vars =="EF.surface":
                var = ["LHTFL.surface.6_hour_ave","SHTFL.surface.6_hour_ave"]
                df1 = xr.open_dataset(dirs[pp]+id+'/'+var[0]+'.nc4')[var[0]].sel(longitude=lon, latitude=lat, method="nearest")
                df2 = xr.open_dataset(dirs[pp]+id+'/'+var[1]+'.nc4')[var[1]].sel(longitude=lon, latitude=lat, method="nearest")
                
                ft = len(df1[:,0,0])
                df = df1[:ft,0,:].rename({'time1': 'time', 'longitude': 'sites'}).rename(vars).copy(deep=True)
                df[:] = np.nan
                df.coords['sites'] = site
                del df['latitude']
                
                df.attrs["short_name"] = "EF"
                df.attrs["long_name"] = "Evaporative Fraction"
                df.attrs["units"] = "Dimensionless"
                
                for ff in range(len(lon)):
                    ft = len(df1[:,ff,ff])
                    df[:ft,ff] = calc_ef(df1[:,ff,ff],df2[:,ff,ff])
                                
                df = df.assign_coords({"lon": ("sites", lon)})
                df = df.assign_coords({"lat": ("sites", lat)})
                
                encoding = {vars: comp}
                df.to_netcdf(path='./data/'+p+'_'+id+'_'+vars+'.nc4',engine="netcdf4",unlimited_dims="time",format="netCDF4",encoding=encoding)

                del var
                del df
                del df1
                del df2              
                
            elif vars =="ME.surface":
                var = ["TMP.surface","SPFH.2_m_above_ground"]
                df1 = xr.open_dataset(dirs[pp]+id+'/'+var[0]+'.nc4')[var[0]].sel(longitude=lon, latitude=lat, method="nearest")
                df2 = xr.open_dataset(dirs[pp]+id+'/'+var[1]+'.nc4')[var[1]].sel(longitude=lon, latitude=lat, method="nearest")
                
                ft = len(df1[:,0,0])
                df = df1[:ft,0,:].rename({'time1': 'time', 'longitude': 'sites'}).rename(vars).copy(deep=True)
                df[:] = np.nan
                df.coords['sites'] = site
                del df['latitude']
                
                df.attrs["short_name"] = "ME"
                df.attrs["long_name"] = "Moist Enthalpy"
                df.attrs["units"] = "J/kg"
                
                for ff in range(len(lon)):
                    ft = len(df1[:,ff,ff])
                    df[:ft,ff] = calc_me(df1[:,ff,ff],df2[:,ff,ff])
                                
                df = df.assign_coords({"lon": ("sites", lon)})
                df = df.assign_coords({"lat": ("sites", lat)})
                
                encoding = {vars: comp}
                df.to_netcdf(path='./data/'+p+'_'+id+'_'+vars+'.nc4',engine="netcdf4",unlimited_dims="time",format="netCDF4",encoding=encoding)

                del var
                del df
                del df1
                del df2
                
print("processing done")

P5 20120101 LCL.surface


KeyboardInterrupt: 